<a href="https://colab.research.google.com/github/jhj1819/Machine-learning/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제3(정한준)

#  [A] 산림 식생 유형 분석

In [133]:
import numpy as np
import pandas as pd

In [134]:
df = pd.read_csv("/content/sample_data/forest.csv")
df.head()

,elevation,aspect,slope,hdist2water,vdist2water,hdist2road,shade_9am,shade_noon,shade_3pm,hdist2fire,w1,w2,w3,s1,s2,cov_type
0,3160,347,13,127,15,4621,196,219,162,3139,1,0,0,0,0,1
1,3100,352,9,170,14,5367,205,225,158,1167,1,0,0,1,0,1
2,2675,31,11,212,20,3360,217,216,133,3072,0,0,1,0,1,1
3,3088,43,8,0,0,1611,222,222,136,674,0,0,1,1,0,1
4,3061,113,22,150,19,696,251,212,78,742,1,0,0,1,0,1


 1) 특성행렬과 레이블 벡터를 만들고 훈련셋(80%)과 테스트셋(20%)으로 분할하시오.

In [135]:
y = df.iloc[:, -1]
X = df.drop('cov_type', axis=1)

In [136]:
y.shape

(7000,)

In [137]:
X.shape

(7000, 15)

In [138]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2,
                                                    stratify =y, random_state=1)

 2) ~6) GridSearchCV(10-fold)를 사용하여 각 모델의 베스트 모델을 구하고 테스트셋의 정확
도를 구하시오. 데이터의 표준화가 필요한 모형은 파이프라인으로 구현하시오.

In [139]:
from sklearn.model_selection import GridSearchCV

 2) 로지스틱회귀 : C = [0.01, 0.1, 1.0, 10, 100]

In [140]:
# 특성 표준화
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [141]:
from sklearn.linear_model import LogisticRegression

C = [0.01, 0.1, 1.0, 10, 100]

logistic_gs = GridSearchCV(
    estimator=LogisticRegression(random_state=1),
    param_grid={'C': C},
    scoring='accuracy',
    cv=10,
    n_jobs=-1
)

logistic_gs.fit(X_train_std, y_train)
print(logistic_gs.best_score_)
print(logistic_gs.best_params_)
print(logistic_gs.score(X_test_std, y_test))


0.6889285714285714
{'C': 100}
0.6871428571428572


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


3) k-NN 분류기 : n_neighbors = [1, 3, 5, 7, 9]

In [142]:
from sklearn.neighbors import KNeighborsClassifier
k = [1,3,5,7]
knn_gs = GridSearchCV(estimator = KNeighborsClassifier(),
                       param_grid={"n_neighbors":k},
                       scoring='accuracy', cv=10, n_jobs=-1)

knn_gs.fit(X_train_std, y_train)
print(knn_gs.best_score_)
print(knn_gs.best_params_)
print(knn_gs.score(X_test_std, y_test))

0.7233928571428571
{'n_neighbors': 1}
0.7314285714285714


4) 결정트리 분류기 : max_depth = [1, 2, ...., 30]

In [143]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

depth_grid = {'max_depth':range(1,31)}

tree_gs = GridSearchCV(estimator = DecisionTreeClassifier(random_state=1),
                       param_grid=depth_grid,
                       scoring='accuracy', cv=10, n_jobs=-1)
tree_gs.fit(X_train_std, y_train)
print(tree_gs.best_score_)
print(tree_gs.best_params_)
print(tree_gs.score(X_test_std, y_test))

0.7282142857142857
{'max_depth': 15}
0.7178571428571429


 5) 랜덤 포레스트 분류기(n_estimators=200) : max_depth = [1, 2, ...., 10]

In [144]:
from sklearn.ensemble import RandomForestClassifier

max_depth = {'max_depth':range(1,11)}

forest = RandomForestClassifier(n_estimators=200, max_leaf_nodes=9, random_state=1, n_jobs=-1)

random_forest_gs = GridSearchCV(
     estimator = forest,
     param_grid = max_depth,
     scoring='accuracy', cv=10, n_jobs=-1)

random_forest_gs.fit(X_train_std, y_train)
print(random_forest_gs.best_score_)
print(random_forest_gs.best_params_)
print(random_forest_gs.score(X_test_std, y_test))

0.6712499999999999
{'max_depth': 5}
0.6692857142857143


 6) AdaBoost 분류기(n_estimators=200, base estimator : 최대 깊이가 9인 결정트리) :
learning_rate = [0.1, 0.2, ..., 1.0]
- 시간이 너무 오래걸려 n_estimators=100, 최대깊이=5로 수정

In [150]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

learning_rate = np.arange(0.1, 1.1, 0.1)
ada_gs = GridSearchCV(
    estimator = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=5),
                                   n_estimators=100, algorithm = "SAMME", random_state=1),
    param_grid =  {"learning_rate": learning_rate},
    scoring = 'accuracy', cv=10, n_jobs=-1)

ada_gs.fit(X_train_std, y_train)
print(ada_gs.best_score_)
print(ada_gs.best_params_)
print(ada_gs.score(X_test_std, y_test))

0.7521428571428571
{'learning_rate': 0.4}
0.7578571428571429


 7) 결과를 정리한 아래와 같은 표를 마크다운으로 작성하시오

   
  

 | 모형     | 최적 초모수     | 테스트 스코어     |
|-------------|-------------|-------------|
| 로지스틱회귀  | C=100   | 0.687   |
| knn   | n_neighbors=1   | 0.731   |
| 트리 | max_depth=15 | 0.718 |
| 랜덤포레스트 | max_depth=5 | 0.669 |
| Ada | learning_rate=0.4 | 0.758 |


# [B] 은행 잔고 예측

In [151]:
balance_df = pd.read_csv('/content/sample_data/balance.csv')
balance_df.shape

(400, 12)

1) 특성행렬과 타겟 벡터를 만들고 분할하시오.(분할 비율은 디폴트 값 사용)

In [152]:
balance_y = balance_df.iloc[:,-1].values # pandas로 쓸 이유 없으므로
balance_X = balance_df.drop('Balance', axis=1)

In [153]:
balance_X_train, balance_X_test, balance_y_train, balance_y_test = train_test_split(balance_X, balance_y, random_state=1)

In [154]:
print(balance_X_train.shape)
print(balance_y_test.shape)

(300, 11)
(100,)


 2) 연속형 특성들 사이의 상관계수를 구하시오. 가장 강한 상관관계를 갖는 두 특성 중 타겟
과 상관계수가 더 큰 특성만 남기고 다른 특성은 분석에서 삭제하시오.

In [155]:
num = ['Income',	'Limit',	'Rating',	'Cards',	'Age',	'Education']

In [156]:
balance_X_train_num = balance_X_train[num]
balance_X_test_num = balance_X_test[num]
balance_X_train_num.corr()

,Income,Limit,Rating,Cards,Age,Education
Income,1.0,0.8,0.8,0.0,0.2,-0.0
Limit,0.8,1.0,1.0,0.0,0.1,-0.0
Rating,0.8,1.0,1.0,0.1,0.1,-0.0
Cards,0.0,0.0,0.1,1.0,0.0,-0.0
Age,0.2,0.1,0.1,0.0,1.0,0.0
Education,-0.0,-0.0,-0.0,-0.0,0.0,1.0


Limit과 Rating 사이의 상관관계가 매우 높음.

In [157]:
# pd.options.display.float_format = '{:.1f}'.format
balance_X_train_num_corr = balance_X_train_num[['Limit', 'Rating']]
balance_X_train_num_corr['Balance'] = balance_y_train
balance_X_train_num_corr.corr()

<ipython-input-157-d174ea0f6693>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_X_train_num_corr['Balance'] = balance_y_train


,Limit,Rating,Balance
Limit,1.0,1.0,0.9
Rating,1.0,1.0,0.9
Balance,0.9,0.9,1.0


Rating : 0.872
Limit : 0.870
> Limit 특성 제거

In [158]:
balance_X_train_num.drop('Limit', axis=1)
balance_X_test_num.drop('Limit', axis=1)

,Income,Rating,Cards,Age,Education
398,37.7,192,1,44,13
125,27.6,195,1,34,15
328,41.2,297,3,54,16
339,149.3,707,1,80,16
172,76.3,344,4,60,18
...,...,...,...,...,...
300,21.8,355,1,50,17
277,41.5,353,2,50,12
289,10.9,296,2,30,9
260,67.9,383,4,63,12


 3) 범주형 특성들을 원-핫 인코딩한 후, 연속형 특성들과 합친 후 표준화하시오.

- 범주형 :Gender	Student	Married Ethnicity

In [159]:
cat = ['Gender', 'Student', 'Married', 'Ethnicity']
balance_X_train_cat = balance_X_train[cat]
balance_X_test_cat = balance_X_test[cat]

In [160]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first', feature_name_combiner='concat')

In [161]:
one_hot_encoder.fit(balance_X_train_cat)

OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

In [162]:
balance_X_train_cat_encoded = pd.DataFrame(one_hot_encoder.transform(balance_X_train_cat),
                                   columns = one_hot_encoder.get_feature_names_out(),
                                   index=balance_X_train_cat.index)
balance_X_train_cat_encoded.head()

,Gender_Male,Student_Yes,Married_Yes,Ethnicity_Asian,Ethnicity_Caucasian
82,1.0,0.0,0.0,0.0,1.0
367,1.0,0.0,0.0,0.0,0.0
179,0.0,0.0,0.0,0.0,1.0
27,1.0,0.0,0.0,0.0,0.0
89,0.0,0.0,0.0,0.0,0.0


In [163]:
balance_X_test_cat_encoded = pd.DataFrame(one_hot_encoder.transform(balance_X_test_cat),
                                   columns = one_hot_encoder.get_feature_names_out(),
                                   index=balance_X_test_cat.index)

합치기

In [164]:
X_train_encoded = pd.concat([balance_X_train_num, balance_X_train_cat_encoded], axis=1)
X_test_encoded = pd.concat([balance_X_test_num, balance_X_test_cat_encoded], axis=1)

표준화

In [165]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [166]:
X_train_std = scaler.fit_transform(X_train_encoded)
X_test_std = scaler.transform(X_test_encoded)

 4) 선형회귀를 실시하여 테스트셋의 결정계수를 구하시오.

In [167]:
from sklearn.linear_model import LinearRegression
linearReg = LinearRegression()

In [168]:
linearReg.fit(X_train_std, balance_y_train)

LinearRegression()

In [169]:
print(linearReg.score(X_train_std, balance_y_train))
print(linearReg.score(X_test_std, balance_y_test))

0.9557279779952305
0.9500447382168364


5) 최대 깊이를 튜닝한 회귀 트리 모형의 결정계수를 구하시오.

In [170]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

depth_grid = {'max_depth':range(1,31)}

tree_gs = GridSearchCV(estimator = DecisionTreeRegressor(random_state=1),
                       param_grid=depth_grid,
                       scoring='r2', cv=10, n_jobs=-1)
tree_gs.fit(X_train_std, balance_y_train)
print(tree_gs.best_score_)
print(tree_gs.best_params_)
print(tree_gs.score(X_test_std, balance_y_test))

0.8990424935186292
{'max_depth': 12}
0.8691303041828885


In [171]:
# 테스트 데이터로 최적 모델 평가
best_model = tree_gs.best_estimator_
r2_test = best_model.score(X_test_std, balance_y_test)
print(r2_test)

0.8691303041828885


 6) 최대 깊이를 튜닝한 랜덤 포레스트(n_estimators=200) 모형의 결정계수를 구하시오.

In [172]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

depth_grid = {'max_depth': range(1, 31)}
rf = RandomForestRegressor(n_estimators=200, random_state=1)

rf_gs = GridSearchCV(estimator=rf,
                     param_grid=depth_grid,
                     scoring='r2',
                     cv=10,
                     n_jobs=-1)

rf_gs.fit(X_train_std, balance_y_train)

print(rf_gs.best_score_)
print(rf_gs.best_params_)

r2_test = rf_gs.score(X_test_std, balance_y_test)
print(r2_test)


0.9253470167810411
{'max_depth': 11}
0.9333971357583419


7) 위의 세 모형 중 어느 모형이 더 나은지 비교하시오.

- 선형회귀 : 0.950
- 회귀트리 : 0.899
- 랜덤 포레스트 : 0.933




In [173]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [174]:
%%shell
jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/hwpark.ipynb"

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/HW3.ipynb to html
[NbConvertApp] Writing 404262 bytes to /content/drive/MyDrive/Colab Notebooks/HW3.html
